### RAG with Tabalar Data


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('Chatbot_rag_v1') \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
!pip install langchain==0.2.17
!pip install langchain_community==0.2.19
!pip install -qU langchain-ollama==0.1.3
!pip install -qU langchain-qdrant==0.1.4

In [ ]:
import os
from dotenv import load_dotenv, dotenv_values

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOllama


### Visualizar e pegar uma amostra dos dados

In [ ]:
spark.sql("Select * from iceberg.silver.tbl_silver_olhovivo").limit(10).show()

In [ ]:
df = spark.sql("""
    Select
    c,
    cl,
    sl,
    lt0,
    lt1,
    qv
    
    from iceberg.silver.tbl_silver_olhovivo """
).limit(10)

df.createOrReplaceTempView("vw_silver_olhovivo")

spark.sql("select * from vw_silver_olhovivo").show()

## Funções Auxiliares

In [ ]:
def describe_table(df, table_name="tbl_bus_posicao"):
    """Coletar metadados da tabela"""
    columns = "\n".join([f"- {f.name}: {f.dataType.simpleString()}" for f in df.schema])
    return f"Tabela: {table_name}\n\nColunas:\n{columns}"

import re

def clear_sql(response):
    """Remove blocos de código markdown"""
    sql = re.sub(r"```sql|```", "", response, flags=re.IGNORECASE).strip()
    return sql


### Carrega modelo (Mistral 7B)

In [ ]:
%run ../common/Common_env_functions.ipynb

In [ ]:
load_dotenv('../.env')
OLLAMA_API_URL = os.getenv("OLLAMA_API_URL")

In [ ]:
llm = ChatOllama(
    model="mistral:latest", 
    base_url=OLLAMA_API_URL,
    temperature=0.3,
    num_predict=200,
    top_k=30,
    top_p=0.9,
    repeat_penalty=1.1
    
) 

### Configurar Promps (roles System, Human)

In [ ]:
# Prompt para gerar SQL (com roles)
prompt_sql = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Você é um especialista em dados. Gere apenas a consulta SQL."),
    HumanMessagePromptTemplate.from_template(
        "Com base na estrutura da tabela abaixo:\n\n{schema}\n\n"
        "Escreva uma consulta SQL (somente a SQL) para responder:\n{question}"
    )
])


In [ ]:
# Prompt para gerar resposta para o usuário (com roles)
prompt_response = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Você é um assistente de dados."),
    HumanMessagePromptTemplate.from_template(
        "Pergunta: {question}\n\nResultado da consulta:\n{result}\n\n"
        "Gere uma resposta clara e amigável para o usuário."
    )
])

### Chatbot com RAG

In [ ]:
def augmented_response(question):
    print(f"\n💬question: {question}")

    #Gerar SQL com role, a partir da pergunta do usuário (Especialista de Dados)
    schema_txt = describe_table(df, "vw_silver_olhovivo")
    sql_chain = prompt_sql | llm 
    sql_result = sql_chain.invoke({"question": question, "schema": schema_txt})    
   
    sql_query = clear_sql(sql_result.content)
    print(f"\n🤖💡 Genereted SQL:\n{sql_query}")

    # Executar SQL gerado pela "role especialista de dados" no Spark
    try:
        result_df = spark.sql(sql_query).toPandas().to_dict(orient="records")
    except Exception as e:
        print(f"❌ Erro na execução da SQL: {e}")
        return

    # Gerar resposta final com role Human (Assistente de Dados)
    response_chain = prompt_response | llm
    response_result = response_chain.invoke({
        "question": question,
        "result": result_df
    })
    
    response = response_result.content.strip()

    print(f"\n🤖 response:\n{response}")

In [ ]:
augmented_response("")